In [17]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from pandas import MultiIndex, Int64Index
# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import optuna
import pandas as pd
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
from sklearn.model_selection import train_test_split
import bokeh



/var/folders/32/zv3nm8_12xb9ytd7vk7nnr6w0000gn/T/ipykernel_38034/3489285944.py:8: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [18]:
#!pip3 install xgboost
#!pip3 install optuna
#!pip install autogluon --upgrade
# !pip install optuna
# !pip install scipy
# !pip install autogluon
# !pip install xgboost
# !python3.9 -m pip install --upgrade pip
#!pip install bokeh==2.0.1
#!pip install jupyter --upgrade


In [19]:
#!conda install mxnet
#import mxnet

In [20]:
df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')

In [21]:
np.random.seed(0)
train, valid = train_test_split(df_train, test_size=0.2)

In [22]:
combined = df_train.drop(['price', 'size_sqft'], axis=1)
X_train = train.drop(['price', 'size_sqft'], axis=1)

X_valid = valid.drop(['price', 'size_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
df_test= df_test.drop(['size_sqft'], axis=1)

metric_reg = 'mean_absolute_error'
label_reg = 'price_per_sqft'
train_data_reg = TabularDataset(X_train) 
valid_data_reg = TabularDataset(X_valid)
test_data_reg = TabularDataset(df_test)
combined = TabularDataset(combined)



Tabular Predictor

In [24]:
predictor = TabularPredictor(label = label_reg, eval_metric = metric_reg)

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {
    #'NN_TORCH': {},
 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}],
 'CAT': {},
 #'NN_MXNET':{},
 'XT':{}
}

time_limit = 30*60  # train various models for ~5 min
num_trials = 3  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}



#excluded_model_types = ['NN_TORCH', 'FASTAI', 'KNN', 'CAT', 'RF']
excluded_model_types = ['FASTAI', 'KNN', 'RF', 'XGB']
predictor_reg = predictor.fit(train_data_reg, excluded_model_types = excluded_model_types
,hyperparameters=hyperparameters
#,tuning_data=valid_data_reg
,hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, presets='best_quality'
)


No path specified. Models will be saved in: "AutogluonModels/ag-20221028_001415/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221028_001415/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    14739
Train Data Columns: 31
Label Column: price_per_sqft
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4525.797460167968, 461.7295308187673, 1811.79782, 967.43626)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
/usr/local/lib/python3.9/site-packages/autogluon/tabular/learner/de

  0%|          | 0/3 [00:00<?, ?it/s]

[1000]	valid_set's l1: 164.253
[2000]	valid_set's l1: 161.893
[3000]	valid_set's l1: 161.227
[4000]	valid_set's l1: 160.888
[5000]	valid_set's l1: 160.828
[6000]	valid_set's l1: 160.73
[7000]	valid_set's l1: 160.593
[8000]	valid_set's l1: 160.873
[9000]	valid_set's l1: 161.053
[1000]	valid_set's l1: 159.013
[1000]	valid_set's l1: 163.775
[2000]	valid_set's l1: 160.857
[3000]	valid_set's l1: 159.705
[4000]	valid_set's l1: 159.273
[5000]	valid_set's l1: 159.574


Fitted model: LightGBMXT_BAG_L1/T1 ...
	-160.5425	 = Validation score   (-mean_absolute_error)
	8.3s	 = Training   runtime
	0.33s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1/T2 ...
	-158.78	 = Validation score   (-mean_absolute_error)
	3.6s	 = Training   runtime
	0.08s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1/T3 ...
	-159.2049	 = Validation score   (-mean_absolute_error)
	9.09s	 = Training   runtime
	0.3s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...


  0%|          | 0/3 [00:00<?, ?it/s]

...

Fitted model: LightGBMXT_BAG_L2/T1 ...
	-157.0127	 = Validation score   (-mean_absolute_error)
	10.54s	 = Training   runtime
	0.59s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L2/T2 ...
	-156.1671	 = Validation score   (-mean_absolute_error)
	15.88s	 = Training   runtime
	0.38s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L2/T3 ...
	-164.3551	 = Validation score   (-mean_absolute_error)
	0.95s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L2 ...


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

[1000]	valid_set's l1: 148.631
[2000]	valid_set's l1: 146.026
[3000]	valid_set's l1: 144.903
[4000]	valid_set's l1: 144.531
[5000]	valid_set's l1: 144.55
[6000]	valid_set's l1: 144.592
[1000]	valid_set's l1: 138.454
[2000]	valid_set's l1: 135.661
[3000]	valid_set's l1: 134.382
[4000]	valid_set's l1: 134.231
[5000]	valid_set's l1: 133.977
[6000]	valid_set's l1: 133.875
[7000]	valid_set's l1: 134.012
[1000]	valid_set's l1: 148.656
[2000]	valid_set's l1: 146.901
[3000]	valid_set's l1: 146.328
[4000]	valid_set's l1: 146.127
[5000]	valid_set's l1: 146.179
[1000]	valid_set's l1: 145.435


	-145.5011	 = Validation score   (-mean_absolute_error)
	65.54s	 = Training   runtime
	1.74s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L2/T3 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-149.4884	 = Validation score   (-mean_absolute_error)
	8.65s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost_BAG_L2/T1 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_fea

In [25]:
#/Users/nishita/Documents/Semester 1/CS5228/project/notebooks/AutogluonModels/ag-20221027_054028
folderName = "AutogluonModels/ag-20221028_001415/"
predictor = TabularPredictor.load(folderName)
predictor.fit_hyperparameters_

{'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}],
 'CAT': {},
 'XT': {}}

In [32]:
predictor.evaluate(valid_data_reg)

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

{'mean_absolute_error': -145.6304617727714,
 'root_mean_squared_error': -292.88972240525686,
 'mean_squared_error': -85784.38949062843,
 'r2': 0.9066739116978073,
 'pearsonr': 0.9522197563763414,
 'median_absolute_error': -74.19214787232931}

In [27]:
#With target encoding and less one hot encoding.
predictor = TabularPredictor.load(folderName)
results = predictor.fit_summary(show_plot=True)

def writePredictionsToFile(modelName, filename, folder = folderName, refit = False):
    if(refit == True):
        modelName = modelName +  '_FULL'    
    y_pred = predictor.predict(test_data_reg, model=modelName)
    yTest = y_pred * df_test_size_sqft
    yTest = pd.DataFrame(yTest, columns=['Predicted'], index=df_test.index)
    yTest.to_csv(f'{folder}/predictions/{filename}.csv')


*** Summary of fit() ***
Estimated performance of each model:
                   model   score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3 -142.068417       8.135790  4147.233197                0.000648           0.317045            3       True         14
1    WeightedEnsemble_L2 -142.819399       1.044720  3722.726048                0.000689           0.312974            2       True          7
2     CatBoost_BAG_L1/T3 -143.111538       0.090473  1497.658844                0.090473        1497.658844            1       True          6
3     CatBoost_BAG_L1/T1 -144.831912       0.068832  1319.087439                0.068832        1319.087439            1       True          4
4     CatBoost_BAG_L2/T3 -145.295000       6.332602  4048.957216                0.042637         191.765848            2       True         13
5   LightGBMXT_BAG_L2/T2 -145.501135       8.025588  3922.734464                

In [28]:
predictor.leaderboard(valid_data_reg)
predictor.evaluate(valid_data_reg)

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

                   model  score_test   score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3 -145.630462 -142.068417       19.243859       8.135790  4147.233197                 0.004866                0.000648           0.317045            3       True         14
1     CatBoost_BAG_L2/T3 -146.081631 -145.295000       14.932324       6.332602  4048.957216                 0.117428                0.042637         191.765848            2       True         13
2    WeightedEnsemble_L2 -146.426780 -142.819399        2.739450       1.044720  3722.726048                 0.006107                0.000689           0.312974            2       True          7
3     CatBoost_BAG_L1/T1 -146.484567 -144.831912        0.185685       0.068832  1319.087439                 0.185685                0.068832        1319.087439            1       True          4
4     CatBoost_BAG_L

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

{'mean_absolute_error': -145.6304617727714,
 'root_mean_squared_error': -292.88972240525686,
 'mean_squared_error': -85784.38949062843,
 'r2': 0.9066739116978073,
 'pearsonr': 0.9522197563763414,
 'median_absolute_error': -74.19214787232931}

In [30]:
writePredictionsToFile(modelName = 'WeightedEnsemble_L2', filename = 'WeightedEnsemble_L2')


/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [31]:
#writePredictionsToFile(modelName = 'LightGBMXT/T2', filename = 'LightGBMXT')
#writePredictionsToFile('RandomForestMSE')
#writePredictionsToFile('LightGBMLarge')
writePredictionsToFile(modelName = 'CatBoost_BAG_L2/T3', filename = 'CatBoost')
##writePredictionsToFile('XGBoost')
#writePredictionsToFile(modelName = 'ExtraTreesMSE', filename = 'ExtraTreesMSE')
#writePredictionsToFile('LGBModel')
writePredictionsToFile(modelName = 'LightGBMXT_BAG_L2/T2', filename = 'LightGBMXT')
#writePredictionsToFile(modelName = 'NeuralNetTorch/a2acf_00004', filename = 'NeuralNetTorch')
#writePredictionsToFile('NeuralNetFastAI')
#writePredictionsToFile('KNeighborsDist')
##writePredictionsToFile('KNeighborsUnif')










/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [ ]:
#predictor.refit_full()

In [ ]:
actual = pd.read_csv('data/example-submission.csv')
    pred = pd.read_csv(f'notebooks/{folder}/predictions/{modelName}.csv')

    actual = actual['Predicted']
    pred = pred['Predicted']


predictor.leaderboard(test_data_reg)